# Continuous Control

---

Code for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
# !pip -q install ./python

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import importlib
import time
import torch

# Import custom agent
import ddpg_agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
#env = UnityEnvironment(file_name='./Reacher_Linux/Reacher.x86_64') # Unity env with Viz
env = UnityEnvironment(file_name='./Reacher_Linux_NoVis/Reacher.x86_64') # Unity env without Viz
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64') # Unity for Udacity workspace

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
print('Shape State Space', states.shape)
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

Watch agent's performance, when it selects actions at random at each timestep

In [ ]:
"""
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
"""

### 4. Train Model using DDPG

In [ ]:
# Initialize Target Parameters as in "The Environment - Introduction"
average_target_score = 30 # goal average target score to reach
n_consecutive_episodes = 100 # consecutive episodes to average over

In [ ]:
importlib.reload(ddpg_agent)

In [ ]:
# initiate agent
agent = ddpg_agent.Agent(state_size=state_size, action_size=action_size, random_seed=2)

In [ ]:
def ddpg(n_episodes=1000, max_t=300, print_every=n_consecutive_episodes, n_actions=action_size):
    scores_deque = deque(maxlen=n_consecutive_episodes)
    scores = []
    for i_episode in range(1, n_episodes+1):
        # Start timing
        time_start_episode = time.time()
        
        # Reset environment
        env_loc = env.reset(train_mode=True)[brain_name] 
        states = env_loc.vector_observations  
        n_agents = states.shape[0] # shape in example env. (20,33)
        
        # Reset agent
        agent.reset()
        # Array to keep track of individual agent scores
        agent_scores_individual = np.zeros(n_agents)
        
        # Loop over time
        for t in range(max_t):
            # Adapted multi-agent actions
            # ----------------------
            # initialize actions array
            actions = np.zeros((n_agents, n_actions))
            # loop over all agents individually
            for i in range(n_agents):
                agent_act = agent.act(states[i]) # return action per current policy
                actions[i] = np.clip(agent_act, -1, 1)
            # ----------------------
            
            # Interact with environment using new actions
            # ----------------------
            env_loc = env.step(actions)[brain_name] # send all actions to the environment
            next_states = env_loc.vector_observations # get next state (for each agent)
            rewards = env_loc.rewards # get reward (for each agent)
            dones = env_loc.local_done # see if episode finished
            # ----------------------
            
            # Adapted multi-agent step:
            # ----------------------
            # Add experience from each agent to replay buffer
            for i in range(n_agents):
                agent.add_to_memory(states[i], actions[i], rewards[i], next_states[i], dones[i])
             
            # Every 20 time steps --> perform network update
            if t % 20 == 0:
                for i in range(10): # as proposed in "Benchmark implementation" 10 fold update
                    agent.multi_agent_step() 
            # ----------------------

            agent_scores_individual += rewards # update the score (for each agent)
            states = next_states # roll over states to next time step
            
            if np.any(dones): # exit loop if episode finished
                break

        # For each episode calculate mean score overall individual agents
        mean_score = np.mean(agent_scores_individual)
          
        # Add mean score to scores
        scores_deque.append(mean_score)
        mean_scores_deque = np.mean(scores_deque)
        scores.append(mean_score)
        
        # Print update after print_every
        if i_episode % print_every == 0:
            time_end_episode = time.time()
            average_time_per_episode = (time_end_episode-time_start_episode)
            print('\rEpisode {}\tAverage Score: {:.2f}\tEpisode Time s: {:.2f}'.format(i_episode, mean_scores_deque, average_time_per_episode))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        # stop training once target score is reached   
        if mean_scores_deque >= average_target_score:
            print('\rEnvironment solved at Episode {}\tAverage Score: {:.2f}'.format(i_episode-n_consecutive_episodes, mean_scores_deque))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_final.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_final.pth')
            break
            
    return scores

In [ ]:
scores = ddpg(n_episodes=1000, max_t=1000, print_every=5, n_actions=action_size)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('./Plots/Average_scores.png')
plt.show()

### 5. Close Environment

In [ ]:
env.close()